# Korean LLM (Large Language Model) fine-tuning on Local environment (Debugging)
---

- 허깅페이스 인증 정보 설정: `huggingface-cli login`
    - https://huggingface.co/join
    - https://huggingface.co/settings/tokens
    

## Overview 

본격적으로 SageMaker 훈련 인스턴스로 훈련을 수행하기 전에 SageMaker Notebook / SageMaker Studio / SageMaker Studio Lab 에서 샘플 데이터로 디버깅을 수행합니다.
물론 온프레미스 환경에서 디버깅을 수행할 수 있다면, 기존 환경과 동일하게 디버깅을 수행하면 됩니다.


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
import torch
import transformers
from datasets import load_dataset, load_from_disk
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast

sys.path.append('../utils')
sys.path.append('../templates')

In [ ]:
%store -r bucket_prefix dataset_prefix s3_data_path

In [ ]:
try:
    dataset_prefix
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] 1번 모듈 노트북을 다시 실행해 주세요.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
lm_dataset = load_from_disk(dataset_prefix)

<br>

## 1. Load Model
---
한정된 GPU 메모리 안에 LLM 모델을 로드하는 것은 매우 어렵습니다. 예컨대 20B의 모델을 로드하려면 fp32 기준으로 80GB 이상의 메모리가 필요하고 fp16 기준으로도 40GB 이상의 GPU 메모리가 필요하며, 파인 튜닝을 수행하는 경우는 이보다 더욱 많은 GPU 메모리가 필요합니다. 이런 경우 4비트 양자화와 LoRA를 사용하면 범용적으로 사용하고 있는 16GB 및 24GB GPU 메모리로도 파인 튜닝이 가능합니다. 현 기준으로는 4비트 양자화를 지원하는 QLoRA 기법이 가장 널리 사용되고 있으며 bitsandbytes를 사용하여 QLoRA를 쉽게 적용할 수 있습니다. QLoRA는 양자화된 파라미터의 분포 범위를 정규 분포 내로 억제하여 정밀도의 저하를 방지하는 4비트 NormalFloat 양자화 양자화를 적용하는 정수에 대해서도 양자화를 적용하는 이중 양자화, 그리고 optimizer state 등의 데이터를 CPU 메모리에 저장하는 페이징 기법을 적용하여 GPU 메모리 사용량을 억제합니다. QLoRA에 대한 자세한 내용은 논문 (https://arxiv.org/pdf/2305.14314.pdf) 을 참조하기 바랍니다.

### Create a bitsandbytes configuration

In [ ]:
from transformers import BitsAndBytesConfig
quant_4bit = True
quant_8bit = False

if quant_4bit:
    nf4_config = BitsAndBytesConfig(
       load_in_4bit=True,
       bnb_4bit_quant_type="nf4",
       bnb_4bit_use_double_quant=True,
       bnb_4bit_compute_dtype=torch.bfloat16
)
else:
    nf4_config = None

In [ ]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "nlpai-lab/kullm-polyglot-12.8b-v2"

# create model dir
model_name = HF_MODEL_ID.split("/")[-1].replace('.', '-')
model_tar_dir = Path(f"/home/ec2-user/SageMaker/models/{model_name}")

In [ ]:
device_map = "auto"

tokenizer = GPTNeoXTokenizerFast.from_pretrained(HF_MODEL_ID)

model = GPTNeoXForCausalLM.from_pretrained(
    model_tar_dir,
    load_in_8bit=True if quant_8bit else False,
    torch_dtype=torch.float16,
    device_map=device_map,
    #cache_dir=cache_dir,
    quantization_config=nf4_config,
)

### Create LoRA config
LoRA 설정에 대한 자세한 내용은 아래를 참조해 주세요.
- https://huggingface.co/docs/peft/conceptual_guides/lora

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)

model = prepare_model_for_kbit_training(model)

lora_r  = 8
lora_alpha = 32
lora_dropout = 0.05
lora_target_modules = ["query_key_value", "xxx"]
    
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

<br>

## 2. Training
---
### Setting Hyperparameters

In [ ]:
train_data = lm_dataset
val_data = None
num_epochs = 1
batch_size = 2

learning_rate = 3e-5
gradient_accumulation_steps = 2
val_set_size = 0
output_dir = 'output'
world_size = 1
ddp = world_size != 1
group_by_length = False

In [ ]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=100,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    bf16=True,
    logging_steps=2,
    optim="paged_adamw_8bit",
    evaluation_strategy="steps" if val_set_size > 0 else "no",
    save_strategy="steps",
    eval_steps=200 if val_set_size > 0 else None,
    save_steps=10,
    output_dir=output_dir,
    load_best_model_at_end=True if val_set_size > 0 else False,
    ddp_find_unused_parameters=False if ddp else None,
    report_to="none",
    group_by_length=group_by_length,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=train_args,
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

### Start Training

In [ ]:
model.config.use_cache = False

# old_state_dict = model.state_dict
# model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
#     model, type(model)
# )

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

train_result = trainer.train()

### Check metrics

In [ ]:
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
#trainer.save_metrics("train", metrics)

### Save fine-tuned model

In [ ]:
trainer.model.save_pretrained(output_dir)

In [ ]:
# Free memory for merging weights
del model
del trainer
torch.cuda.empty_cache()